# Exemple de compter le nombre d'occurence de mots de plus de trois lettres

# 0) on vérifie la connection au cluster

In [14]:
sc

In [15]:
spark

# 1) on lit un fichier dans une rdd

In [16]:
file = sc.textFile("/home/spark/spark/README.md")

# 2) on définit une fonction à appliquer sur chaque ligne
=> Ici on splitte sur les mots

In [18]:
splitter= lambda ligne: ligne.split(" ")

In [21]:
mots = file.flatMap(lambda line : splitter(line))

# 3) on va filtrer sur les mots qui ont plus de trois caractère
=> choix arbitraire

In [22]:
mots_de_trois_caracteres = mots.filter(lambda mot : len(mot)>3)


# 4) on créé une rdd en mode clef valeur en retournant un tuple

In [23]:
clef_valeur              = mots_de_trois_caracteres.map(lambda mot : (mot,1))

# 5) on récupère le résultat au niveau du driver, et on réduit avec l'opérateur add

In [24]:
from operator import add 

In [25]:
denombrement_long_mots   = clef_valeur.reduceByKey(add)

# 6) ici on est à la fin du DAG, et on exécute une action pour l'activer

In [26]:
type(denombrement_long_mots)

pyspark.rdd.PipelinedRDD

# 6.1) action de récupérer des valeurs

In [27]:
denombrement_long_mots.take(10)

[('Apache', 1),
 ('Spark', 16),
 ('provides', 1),
 ('high-level', 1),
 ('APIs', 1),
 ('Scala,', 1),
 ('Java,', 1),
 ('optimized', 1),
 ('engine', 1),
 ('supports', 2)]

# 6.2) action de compter

In [28]:
mots.count()

569

# 6.3) action de trier par valeur décroissante

In [29]:
denombrement_long_mots.sortBy(lambda x: x[1], ascending=False).take(10)

[('Spark', 16),
 ('using', 5),
 ('Please', 4),
 ('with', 4),
 ('also', 4),
 ('build', 4),
 ('including', 4),
 ('documentation', 3),
 ('example', 3),
 ('Hadoop', 3)]

# 6.4) action de supprimer des lignes

In [33]:
mots_a_eliminier = ["using", "Please"]
denombrement_long_mots.filter(lambda x: x not in mots_a_eliminier).sortBy(lambda x: x[1], ascending=False).take(10)

[('Spark', 16),
 ('using', 5),
 ('Please', 4),
 ('also', 4),
 ('including', 4),
 ('build', 4),
 ('with', 4),
 ('documentation', 3),
 ('general', 3),
 ('example', 3)]

# 6.5) action de lister les mots par longueur

In [41]:
# l'astuce consiste à retourner une liste comme valeur
inversion_clef_valeur = denombrement_long_mots.map(lambda k_v : (k_v[1], [k_v[0]]))

In [42]:
# ainsi on peut concaténer les listes
inversion_clef_valeur.reduceByKey(lambda x,y: x+y).take(10)

[(16, ['Spark']),
 (2,
  ['supports',
   'Interactive',
   'Shell',
   'following',
   '1000:',
   'Python',
   'programs',
   './bin/run-example',
   'SparkPi',
   'examples',
   'locally',
   'class',
   'tests',
   'guidance',
   'Hadoop,',
   'refer',
   'particular',
   'Hive',
   'This',
   '`examples`',
   'that',
   'cluster',
   'detailed',
   'building',
   'Python,',
   'Scala',
   'shell:',
   'command,',
   'which',
   'should',
   'return']),
 (4, ['Please', 'with', 'also', 'build', 'including']),
 (1,
  ['<class>',
   'will',
   'locally.',
   'MASTER',
   'environment',
   'running',
   'submit',
   'cluster.',
   'mesos://',
   '"yarn"',
   'thread,',
   '"local[N]"',
   'threads.',
   'MASTER=spark://host:7077',
   'Many',
   'given.',
   'Running',
   'Tests',
   'first',
   'requires',
   '[building',
   '[run',
   'tests](http://spark.apache.org/developer-tools.html#individual-tests).',
   'Versions',
   'core',
   'talk',
   'protocols',
   'same',
   'your',
   '